In [11]:
import os
import pandas as pd

DATAPATH = "../data"

In [12]:
smi = pd.read_csv(os.path.join(DATAPATH, "dilismiles.csv"))
padel = pd.read_csv(os.path.join(DATAPATH, "dili_padel_2d.csv"))

/tmp/ipykernel_2731979/1034768844.py:2: DtypeWarning: Columns (423,424,425,426,427,428,429,430,431,477,478,479,480,481,482,483,484,485,540,541,542,543,544,545,546,547,548,603,604,605,606,607,608,609,610,611,1012,1013,1050,1052,1074,1083,1084,1121,1123,1145,1217,1218,1219,1220,1222,1223,1224,1226,1227,1229) have mixed types. Specify dtype option on import or set low_memory=False.
  padel = pd.read_csv(os.path.join(DATAPATH, "dili_padel_2d.csv"))


In [13]:
#compare that the order of molecules remains the same between both files
# Extract the 'molecule' column from each DataFrame
dilismiles_molecules = smi['col_smiles'].tolist()
dilipadel_molecules = padel['CommonName'].tolist()


# Compare molecule order
def compare_molecule_order(dilismiles_molecules, dilipadel_molecules):
    # Check if the number of molecules is the same in both files
    if len(dilismiles_molecules) != len(dilipadel_molecules):
        print("Error: The number of molecules is not the same in both files.")
        return

    print("Molecule order is the same in both files.")

# Call the function with your molecule lists
compare_molecule_order(dilismiles_molecules, dilipadel_molecules)

Molecule order is the same in both files.


In [14]:
smi["outcome"] = padel["Outcome"]
smi["CommonName"]= padel["CommonName"]
smi.rename(columns={"col_smiles":"smiles"}, inplace=True)

In [15]:
from rdkit import Chem
st_dmi = []
smi.to_csv(os.path.join(DATAPATH,"dilismiles_with_outcome.csv"), index=False)

In [16]:
##compare that the molecule order is the same based on smiles 
import pandas as pd
from rdkit import Chem

# Load DrugBank_DDI_interactions.xls file
xls_file = pd.ExcelFile('../data/DrugBank_DDI_interactions.xls')
sheet_name = '003 - final'  # Replace with the actual sheet name
ddi_df = xls_file.parse(sheet_name)

# Load CSV files
common_names_df = pd.read_csv('../data/dilismiles_with_outcome.csv')


# Check correlation based on both CommonName and col_smiles
merged_df = pd.merge(ddi_df, common_names_df, left_on=['Compound', 'Smiles'], right_on=['CommonName', 'smiles'], how='inner')

# Print the relevant columns
print(merged_df[['Compound', 'CommonName']])

              Compound         CommonName
0      Aurothioglucose    Aurothioglucose
1    DemecariumBromide  DemecariumBromide
2     Dextromethorphan   Dextromethorphan
3           Ergotamine         Ergotamine
4       Beclomethasone     Beclomethasone
..                 ...                ...
583        Myoinositol        Myoinositol
584        Dimercaprol        Dimercaprol
585       Mitoxantrone       Mitoxantrone
586         Disulfiram         Disulfiram
587       Deferoxamine       Deferoxamine

[588 rows x 2 columns]


# Data Analysis

In [17]:
##compare our dataset with TDC dili dataset
import pandas as pd
from rdkit import Chem
from rdkit.Chem import PandasTools

##TDC dataset
dataset1 = pd.read_csv('../data/dili_tdc_dataset.csv')
##Our dataset
dataset2 = pd.read_csv('../data/dilismiles.csv')

# Convert SMILES strings to RDKit molecule objects
PandasTools.AddMoleculeColumnToFrame(dataset1, 'Drug', 'Molecule1')
PandasTools.AddMoleculeColumnToFrame(dataset2, 'col_smiles', 'Molecule2')

# Find the intersection of SMILES
overlap_smiles = set(dataset1['Drug']).intersection(set(dataset2['col_smiles']))

# Calculate the number of drugs in each dataset
num_drugs_paper = len(set(dataset2['col_smiles']))
num_drugs_tdc = len(set(dataset1['Drug']))

# Calculate the number of common drugs and the percentage of overlap
num_common_drugs = len(overlap_smiles)
percent_overlap = (num_common_drugs / num_drugs_paper) * 100

print(f"Number of drugs in paper's dataset: {num_drugs_paper}")
print(f"Number of drugs in TDC's dataset: {num_drugs_tdc}")
print(f"Number of common drugs: {num_common_drugs}")
print(f"Percentage of common drugs relative to paper's dataset: {percent_overlap:.2f}%")

# Convert the set of overlapping SMILES to a DataFrame
overlap_df = pd.DataFrame(list(overlap_smiles), columns=['Overlapping SMILES'])

overlap_df.head()

Number of drugs in paper's dataset: 587
Number of drugs in TDC's dataset: 475
Number of common drugs: 67
Percentage of common drugs relative to paper's dataset: 11.41%


,Overlapping SMILES
0,CCCOc1ccc2nc(NC(=O)OC)[nH]c2c1
1,NC(CO)(CO)CO
2,CCOc1ccc(NC(C)=O)cc1
3,S=c1nc[nH]c2nc[nH]c12
4,Cc1ccnc2c1NC(=O)c1cccnc1N2C1CC1


In [18]:
## Delete repeded smiles from the TDC's dataset

import pandas as pd

# Read the TDC dataset
tdc_dataset = pd.read_csv('../data/dili_tdc_dataset.csv')

# Read paper dataset
paper_dataset = pd.read_csv('../data/dilismiles.csv')

# Extract the common smiles between the two datasets
common_smiles = set(tdc_dataset['Drug']).intersection(set(paper_dataset['col_smiles']))

# Remove rows with common smiles from the TDC dataset
filtered_tdc_dataset = tdc_dataset[~tdc_dataset['Drug'].isin(common_smiles)]

# Save the result to a new file
filtered_tdc_dataset.to_csv('../data/filtered_tdc_dataset.csv', index=False)




In [19]:
##get the ratio of positive and negative outcomes
import pandas as pd

# Read the CSV file into a DataFrame
df = pd.read_csv('../data/dili_padel_2d.csv')

# Count the occurrences of positive (1) and negative (0) outcomes in the 'outcome' column
outcome_counts = df['Outcome'].value_counts()

# Print the result
total_samples = len(df)
print("Number of Negative (0) outcomes:", outcome_counts[0])
print("Number of Positive (1) outcomes:", outcome_counts[1])
print("Total number of samples:", total_samples)

# Calculate and print the ratio of negative (0) and positive (1) outcomes
outcome_ratio = df['Outcome'].value_counts(normalize=True)
print("\nRatio of Negative (0) outcomes:", outcome_ratio[0])
print("Ratio of Positive (1) outcomes:", outcome_ratio[1])


Number of Negative (0) outcomes: 194
Number of Positive (1) outcomes: 394
Total number of samples: 588

Ratio of Negative (0) outcomes: 0.3299319727891156
Ratio of Positive (1) outcomes: 0.6700680272108843


/tmp/ipykernel_2731979/1141917293.py:5: DtypeWarning: Columns (423,424,425,426,427,428,429,430,431,477,478,479,480,481,482,483,484,485,540,541,542,543,544,545,546,547,548,603,604,605,606,607,608,609,610,611,1012,1013,1050,1052,1074,1083,1084,1121,1123,1145,1217,1218,1219,1220,1222,1223,1224,1226,1227,1229) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('../data/dili_padel_2d.csv')
